In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv
/kaggle/input/jigsaw-agile-community-rules/train.csv
/kaggle/input/jigsaw-agile-community-rules/test.csv


In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)

2025-07-30 10:01:32.493925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753869692.700094      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753869692.755157      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/train.csv')
df_test = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/test.csv')

df = df[['body', 'rule_violation']].dropna()
df_train, df_val = train_test_split(df, test_size=0.1, stratify=df['rule_violation'], random_state=42)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier
import pandas as pd
from scipy.sparse import hstack

for df in [df_train, df_test]:
    df['body_len'] = df['body'].apply(len)
    df['num_links'] = df['body'].str.count(r"http\S+")
    df['num_exclaims'] = df['body'].str.count('!')
    df['num_upper'] = df['body'].str.count(r'[A-Z]{2,}')

meta_features = ['body_len', 'num_links', 'num_exclaims', 'num_upper']

vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(4, 6), max_features=50000)
X_tfidf = vectorizer.fit_transform(df_train['body'])
X_test_tfidf = vectorizer.transform(df_test['body'])

X = hstack([X_tfidf, df_train[meta_features].values])
X_test = hstack([X_test_tfidf, df_test[meta_features].values])

model = LGBMClassifier(n_estimators=500, random_state=42)
model.fit(X, df_train['rule_violation'])

y_pred = model.predict_proba(X_test)[:, 1]

submission = pd.DataFrame({
    'row_id': df_test['row_id'],
    'rule_violation': y_pred
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv oluşturuldu.")


[LightGBM] [Info] Number of positive: 928, number of negative: 898
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 70609
[LightGBM] [Info] Number of data points in the train set: 1826, number of used features: 3702
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508215 -> initscore=0.032862
[LightGBM] [Info] Start training from score 0.032862
✅ submission.csv oluşturuldu.


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")
